- http://keunwoochoi.blogspot.com/2016/03/2.html
- http://www.rex-ai.info/docs/AI_Example_CNN_speech_recognize
- https://www.youtube.com/watch?v=oltGIc4uo5c
- https://youdaeng-com.tistory.com/5

In [4]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn # 인공 신경망 모델들 모아놓은 모듈
import torch.nn.functional as F #그중 자주 쓰이는것들을 F로
from torchvision import transforms, datasets
import cv2
from torchvision import transforms, datasets
import pandas as pd
import os
from glob import glob

if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
#DEVICE = torch.device('cpu')
print('Using Pytorch version : ',torch.__version__,' Device : ',DEVICE)

Using Pytorch version :  1.10.0  Device :  cuda


# SVD 문장 데이터에서 Feature 추출
- mfcc

In [9]:
import numpy as np
import librosa, librosa.display 
import matplotlib.pyplot as plt
#window sizde : FFT를 할때 참조할 그래프 길이 ( 프레임 하나당 sample 수 )
#자연어 처리에서는 25ms 사용. https://ahnjg.tistory.com/93
#초당 50000hz 중 1250개씩 윈도우 사이즈로 사용.
sr=50000
win_length =  np.int64(50000/40) # 1250
n_fft= win_length # WINDOWS SIZE중 사용할 길이. WINDOW SIZE가 넘어가면 나머지 것들은 zero padding
hop_length= np.int64( np.ceil(win_length/4) ) #  얼마만큼 시간 주기(sample)를 이동하면서 분석을 할 것인지. 일반적으로 window size의 1/4
#또는 10ms만큼으로 한다고 한다.
#hop_length가 mfcc의 frame수를 결정한다.

pathology_sig=[]
healthy_sig=[]

pathology=[]
healthy=[]


#PATHOLOGY DATA
for audio_path in os.listdir('../voice_data/disorder/export/'):
    sig, sr = librosa.load('../voice_data/disorder/export/'+audio_path, sr=50000)# 논문에서 f_s = 50 000HZ
    pathology_sig.append(sig)
    MFCCs = librosa.feature.mfcc(sig, sr, win_length=win_length ,n_fft=n_fft, hop_length=hop_length, n_mfcc=13)
    pathology.append(MFCCs)
    

#Healthy data
for audio_path in os.listdir('../voice_data/healthy/export/'):
    sig, sr = librosa.load('../voice_data/healthy/export/'+audio_path, sr=50000)# 논문에서 f_s = 50 000HZ
    healthy_sig.append(sig)
    MFCCs = librosa.feature.mfcc(sig, sr, win_length=win_length ,n_fft=n_fft, hop_length=hop_length, n_mfcc=13)
    healthy.append(MFCCs)
    
print("pathology data 수 : ",len(pathology))
print("healthy data 수 : ",len(healthy))


path_max=max([ len(samples) for samples in pathology_sig])
healthy_max=max([ len(samples) for samples in healthy_sig])
print("가장 긴 path sample :" ,path_max)
print("가장 긴 healthy sample :" ,healthy_max)

pathology data 수 :  852
healthy data 수 :  632
가장 긴 path sample : 317967
가장 긴 healthy sample : 207017


In [10]:
print(path_max/sr,"초")
print(healthy_max/sr,"초")

6.35934 초
4.14034 초


# 결과 확인
- 1 row당 1 frame으로

In [3]:
len(healthy[0][2]) #1번 : 파일. 2번:mfcc
headers = "mfcc1 mfcc2 mfcc3 mfcc4 mfcc5 mfcc6 mfcc7 mfcc8 mfcc9 mfcc10 mfcc11 mfcc12 mfcc13".split()
pd.DataFrame(healthy[0].T,columns=headers)



,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13
0,-664.554626,42.987347,19.686474,17.893703,5.439661,15.109371,25.698231,15.588242,12.777233,12.864212,5.049675,9.790856,18.775772
1,-661.093262,49.232384,25.329205,19.615353,3.622412,11.143848,20.681862,11.148173,10.953643,11.828960,5.742882,12.660656,19.052067
2,-648.416748,59.700802,29.251112,24.551905,5.502740,8.653681,17.301186,9.595350,11.883837,13.102000,3.480697,4.985497,9.246387
3,-641.470398,64.023300,33.875042,33.504829,10.996720,10.126940,20.400213,15.367504,13.606176,16.794329,5.960305,6.227929,16.133617
4,-640.913208,65.740158,37.212051,34.325729,11.286431,10.338777,17.450600,13.628424,13.935904,19.616440,8.325340,10.149255,23.181538
...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,-583.004944,103.957191,27.952499,24.078373,17.805784,23.063253,9.110998,1.523905,4.881299,12.015347,11.527961,-1.548047,1.509847
322,-586.246582,101.427971,23.151981,18.545017,17.543690,25.053160,12.090671,5.663185,11.320033,20.145441,22.497692,10.250481,8.224429
323,-583.195374,100.612839,25.392967,22.381254,10.108490,20.151932,11.551888,2.375783,9.401663,18.179150,18.509977,5.336875,2.377341
324,-591.548950,96.951584,27.558010,26.244709,9.731691,15.010460,17.244743,8.297909,10.048749,12.094749,5.907953,2.231759,2.409741


In [12]:
del pathology
del healthy
del pathology_sig
del healthy_sig

# 데이터 나누기
- pathology : 852 ,  healthy 수 :  632  총 1484
- train :  1187 , test : 297

In [13]:
#1. train, test 나누기
import os
import random #데이터 shuffle 사용
from glob import glob


pathology = glob('../voice_data/disorder/export/*.wav')
healthy = glob('../voice_data/healthy/export/*.wav')
print("Pathology : ",len(pathology))
print("Healthy: ",len(healthy))


X = pathology+healthy # mfcc 데이터 합
print("총 데이터수 : ",len(X))
Y = {} # 라벨
for idx,x in enumerate(X):
    if idx<852:
        Y[x]="pathology"
    else:
        Y[x]="healthy"

#train 1187   test: 297
random.shuffle(X)
DATA_TRAINING_PATH = X[:1186]
DATA_TEST_PATH = X[1187:]

print("훈련 셋 : ",len(DATA_TRAINING_PATH))
print("테스트 셋 : ",len(DATA_TEST_PATH))

Pathology :  852
Healthy:  632
총 데이터수 :  1484
훈련 셋 :  1186
테스트 셋 :  297


In [14]:
Y

{'../voice_data/disorder/export\\101-phrase.wav': 'pathology',
 '../voice_data/disorder/export\\1037-phrase.wav': 'pathology',
 '../voice_data/disorder/export\\1038-phrase.wav': 'pathology',
 '../voice_data/disorder/export\\1039-phrase.wav': 'pathology',
 '../voice_data/disorder/export\\1040-phrase.wav': 'pathology',
 '../voice_data/disorder/export\\1041-phrase.wav': 'pathology',
 '../voice_data/disorder/export\\1042-phrase.wav': 'pathology',
 '../voice_data/disorder/export\\1043-phrase.wav': 'pathology',
 '../voice_data/disorder/export\\1044-phrase.wav': 'pathology',
 '../voice_data/disorder/export\\1045-phrase.wav': 'pathology',
 '../voice_data/disorder/export\\1046-phrase.wav': 'pathology',
 '../voice_data/disorder/export\\1047-phrase.wav': 'pathology',
 '../voice_data/disorder/export\\1048-phrase.wav': 'pathology',
 '../voice_data/disorder/export\\1049-phrase.wav': 'pathology',
 '../voice_data/disorder/export\\105-phrase.wav': 'pathology',
 '../voice_data/disorder/export\\1050-phra

# 데이터 정의
- 추가적으로 데이터의 크기를 맞춰주기 위해 3초로 padding 및 truncate 실시 https://sequencedata.tistory.com/25 FixAudioLength
- 논문에서는 400frame으로 설정.
- 전처리 방법 결정.


In [26]:
from torch.utils.data import Dataset, DataLoader

classes = ["pathology","healthy"]
sr=50000
win_length =  np.int64(50000/40) # 1250
n_fft= win_length # WINDOWS SIZE중 사용할 길이. WINDOW SIZE가 넘어가면 나머지 것들은 zero padding
hop_length= np.int64( np.ceil(win_length/4) ) #  얼마만큼 시간 주기(sample)를 이동하면서 분석을 할 것인지. 일반적으로 window size의 1/4
#또는 10ms만큼으로 한다고 한다.
#hop_length가 mfcc의 frame수를 결정한다.



class svd_dataset(Dataset):
    def __init__(self,data_path_list,classes,transform=None):
        #클래스에서 사용할 인자를 받아 인스턴스 변수로 저장하는 일을 한다.
        #예를들면, 이미지의 경로 리스트를 저장하는 일을 하게 된다.
        self.path_list = data_path_list
        self.label = svd_dataset.get_label(data_path_list)
        self.classes=classes
        self.transform=transform
    
    @classmethod
    def get_label(cls,data_path_list):
        label_list=[]
        for idx in data_path_list:
            label_list.append(Y[idx])
        #print(label_list)
        return label_list
    
    def __len__(self):
        return len(self.path_list)
        #데이터 셋의 길이를 정수로 반환한다. 
        
    def __getitem__(self, idx):
        """
        학습에 사용할 이미지를 반환하면 된다. 이 함수는 인풋으로 index를 받는다. 
        path_list에서 index에 해당하는 이미지를 읽으면 된다. 
        이때 텐서형태의 인풋이 들어오기때문에 리스트로 변환하여준다.(굳이 안해도됨)
        반환값으로는 경로로부터 읽어들여 transform을 적용한 이미지 파일과 정답에 해당하는 클래스를 반환하면 된다.
        이때, 클래스를 숫자로 반환하여야 한다. 
        예를들면 Good: 0 Bad:1. 클래스 리스트의 index로 반환하게 구현하였다.
        """
        #mfcc = X[self.path_list[idx]].T
        sig, sr = librosa.load(self.path_list[idx], sr=50000)# 논문에서 f_s = 50 000HZ
        
        #mfcc 500 FRAME이 되도록 패딩.
        length = 156500
        if length < len(sig):
            data = sig[:length]
        elif length > len(sig):
            data = np.pad(sig, (0, length - len(sig)), "constant")
        
        MFCCs = librosa.feature.mfcc(data, sr, win_length=win_length ,n_fft=n_fft, hop_length=hop_length, n_mfcc=13)
        MFCCs=MFCCs.T
        
        
        
        if self.transform:
            image=self.transform(MFCCs).type(torch.float32)# 데이터 0~1 정규화
        else:
            MFCCs = torch.from_numpy(MFCCs).type(torch.float32)# 이미지 0~1 정규화
            #MFCCs = MFCCs.permute(2, 0, 1)
        return MFCCs, float(self.classes.index(self.label[idx]))
    

# 데이터 로더

In [27]:
#3. 하이퍼 파라미터
BATCH_SIZE =  32 #한 배치당 32개 음성데이터
EPOCHS = 50 # 전체 데이터 셋을 50번 반복

In [28]:
#DATA LOADER 함수가 BATCH_size 단위로 분리해 지정.
train_loader = torch.utils.data.DataLoader(dataset = 
                                           svd_dataset(
                                               DATA_TRAINING_PATH,
                                               classes,
                                               #transform = transforms.ToTensor()#이걸 composed로 고쳐서 전처리 하도록 수정.
                                           ),
                                           batch_size = BATCH_SIZE,
                                           shuffle = True,
                                           ) # 순서가 암기되는것을 막기위해.


test_loader = torch.utils.data.DataLoader(dataset = 
                                           svd_dataset(
                                               DATA_TEST_PATH,
                                               classes,
                                               #transform = transforms.ToTensor()
                                           ),
                                           batch_size = BATCH_SIZE,
                                           shuffle = False,) # 순서가 암기되는것을 막기위해.



# 데이터 확인

In [35]:
# 4. 데이터 확인하기
for (X_train,Y_train) in train_loader:
    print("X_train : ",X_train.size(),'type:',X_train.type())
    print("Y_train : ",Y_train.size(),'type:',Y_train.type())
    break
    
X_train[0]

X_train :  torch.Size([32, 501, 13]) type: torch.FloatTensor
Y_train :  torch.Size([32]) type: torch.DoubleTensor


tensor([[-5.5629e+02,  5.8506e+01,  4.0775e+01,  ..., -5.9635e+00,
          5.1436e-01,  9.5498e+00],
        [-5.3810e+02,  6.4540e+01,  3.8429e+01,  ...,  5.5907e-01,
         -9.8395e-01,  5.6469e+00],
        [-5.1659e+02,  6.3130e+01,  3.7533e+01,  ...,  6.9118e+00,
         -3.4456e+00,  1.0762e+01],
        ...,
        [-6.0682e+02,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-6.0682e+02,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-6.0682e+02,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]])

# CNN